<a href="https://colab.research.google.com/github/zelal-Eizaldeen/project2_Walmart_Price_Prediction/blob/main/project3_3113_zelalae2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# To remove punctuation and numbers
import re
import pandas as pd
import numpy as np
import warnings

#Vis
import matplotlib.pyplot as plt
import seaborn as sns

#ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
import statsmodels.api as sm

# Import BeautifulSoup for Removing HTML Markup
from bs4 import BeautifulSoup
# Import the stop word list
from nltk.corpus import stopwords



In [3]:
# Suppress warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


##Goal
Your goal is to achieve an AUC score of at least 0.986 across all five test data splits.

## Resources:
- Kaggle: https://www.kaggle.com/c/word2vec-nlp-tutorial
- What we have tried 1: https://campuswire.com/c/GB46E5679/feed/785

##Libraries And Packages


In [5]:
# Removing HTML Markup: The BeautifulSoup Package
!pip install BeautifulSoup4


In [6]:
# A stop word list from the Python Natural Language Toolkit (NLTK)
import nltk
nltk.download('stopwords')  # Download text data sets, including stop words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
DATA_DIR='/content/drive/MyDrive/MastersDegree/CS598 PSL/Assignments/Projects/Project3/Data/F24_Proj3_data'

##**Load the Data**

In [8]:
train = pd.read_csv(f"{DATA_DIR}/split_2/train.csv") #quoting=3 tells Python to ignore doubled quotes, otherwise you may encounter errors trying to read the file.

In [ ]:

print(train["review"][0])


Naturally in a film who's main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the relationships and interactions natural showing that you do not need floods of tears to show emotion, screams to show fear, shouting to show dispute or violence to show anger. Naturally Joyce's short story lends the film a ready made structure as perfect as a polished diamond, but the small changes Huston makes such as the inclusion of the poem fit in neatly. It is truly a masterpiece of tact, subtlety and overwhelming beauty.


In [9]:
test = pd.read_csv(f"{DATA_DIR}/split_2/test.csv") #quoting=3 tells Python to ignore doubled quotes, otherwise you may encounter errors trying to read the file.

In [ ]:
test

,id,review,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_1527,embedding_1528,embedding_1529,embedding_1530,embedding_1531,embedding_1532,embedding_1533,embedding_1534,embedding_1535,embedding_1536
0,598,"Now, Throw Momma from the Train was not a grea...",-0.013006,0.069379,-0.111694,0.005056,-0.043846,-0.076886,0.032729,0.033685,...,0.011099,0.024744,0.002237,0.040212,-0.003906,0.003135,0.011075,0.005938,-0.011499,-0.025222
1,12048,"( HR is what Himesh is called in the movie, I ...",-0.008011,0.045142,-0.015509,0.008454,0.010628,-0.030907,0.036003,0.021076,...,-0.001441,0.016769,0.004113,-0.013093,-0.018638,0.016478,-0.001745,0.008412,0.011929,-0.029799
2,40908,Everything is idyllic in Suburbia when the lit...,-0.021345,0.066069,-0.053891,0.034255,-0.007723,-0.007940,0.049389,0.032736,...,0.011167,0.001241,0.010713,-0.003246,0.003360,-0.014781,0.015568,0.008170,-0.012951,-0.057064
3,33283,A powerfully wonderful movie. You are held in ...,-0.021834,0.039822,-0.041830,0.063382,0.001213,-0.009429,0.034238,0.053831,...,-0.003172,-0.005201,-0.022103,0.029854,0.004157,-0.015735,0.001184,0.000577,0.024405,-0.003064
4,40600,"I remember reading all the horrible, horrible ...",-0.029196,0.077836,-0.038469,-0.009751,-0.021623,-0.028690,0.027791,0.016452,...,-0.000343,0.003688,-0.005005,-0.006681,0.000428,0.033017,-0.005223,0.014331,0.002518,-0.021595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,14480,Where Da Vinci code introduced us to Dr. Rober...,-0.041165,0.040130,-0.049036,-0.012007,-0.005579,-0.004045,-0.001575,0.031974,...,0.018706,-0.008356,-0.016777,-0.012117,-0.009049,0.010518,0.018990,0.000415,0.037256,-0.008596
24996,7634,"I haven't seen Ishtar, but I did have the misf...",-0.077530,0.062157,-0.046964,0.005665,-0.004714,-0.069126,0.058468,0.028106,...,-0.002066,0.020728,-0.019728,0.017230,0.008846,-0.016398,-0.025032,0.025301,0.022790,-0.041788
24997,3620,Peter Segal's 1995 commercial hit & now cult-c...,-0.008801,0.021090,-0.071936,0.021405,0.017185,-0.044679,0.001035,0.034280,...,0.027167,0.005666,-0.031939,-0.002144,-0.045309,-0.030094,0.023296,0.022418,0.073872,-0.003846
24998,24858,The reviewer in Variety said this was \overwri...,-0.044322,0.079591,-0.030821,0.024228,-0.032444,-0.018616,0.036316,-0.003181,...,0.010812,0.004124,0.029984,0.019636,-0.007051,0.007365,0.019545,0.009903,0.009910,-0.031240


In [ ]:
# This should be remove before submission
test_y = pd.read_csv(f"{DATA_DIR}/split_1/test_y.csv") #quoting=3 tells Python to ignore doubled quotes, otherwise you may encounter errors trying to read the file.

In [ ]:
test_y

,id,sentiment
0,598,1
1,12048,0
2,40908,1
3,33283,1
4,40600,0
...,...,...
24995,14480,1
24996,7634,0
24997,3620,1
24998,24858,0


##Data Cleaning and Text Preprocessing

##Removing HTML Markup: The BeautifulSoup Package

In [ ]:
# Initialize the BeautifulSoup object on a single movie review
example1 = BeautifulSoup(train["review"][0])

# Print the raw review and then the output of get_text(), for
# comparison
print(train["review"][1])
print(example1.get_text())

Afraid of the Dark left me with the impression that several different screenplays were written, all too short for a feature length film, then spliced together clumsily into this Frankenstein's monster.<br /><br />At his best, the protagonist, Lucas, is creepy. As hard as it is to draw a bead on the secondary characters, they're far more sympathetic.<br /><br />Afraid of the Dark could have achieved mediocrity had it taken just one approach and seen it through -- and had it made Lucas simply psychotic and confused instead of ghoulish and off-putting. I wanted to see him packed off into an asylum so the rest of the characters could have a normal life.
Naturally in a film who's main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the re

**Remove punctuation and numbers**

In [ ]:
# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print(letters_only)

Naturally in a film who s main themes are of mortality  nostalgia  and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones  However there is a craftsmanship and completeness to the film which anyone can enjoy  The pace is steady and constant  the characters full and engaging  the relationships and interactions natural showing that you do not need floods of tears to show emotion  screams to show fear  shouting to show dispute or violence to show anger  Naturally Joyce s short story lends the film a ready made structure as perfect as a polished diamond  but the small changes Huston makes such as the inclusion of the poem fit in neatly  It is truly a masterpiece of tact  subtlety and overwhelming beauty 


## Convert the reviews to lower case and split them into individual words (called "tokenization" in NLP lingo):

In [ ]:
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()

## Frequently occurring words that don't carry much meaning. Such words are called "stop words";
- import a stop word list from the Python Natural Language Toolkit (NLTK)

In [ ]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# Remove stop words from "words"
words = [w for w in words if not w in stopwords.words("english")]
print(words)

['naturally', 'film', 'main', 'themes', 'mortality', 'nostalgia', 'loss', 'innocence', 'perhaps', 'surprising', 'rated', 'highly', 'older', 'viewers', 'younger', 'ones', 'however', 'craftsmanship', 'completeness', 'film', 'anyone', 'enjoy', 'pace', 'steady', 'constant', 'characters', 'full', 'engaging', 'relationships', 'interactions', 'natural', 'showing', 'need', 'floods', 'tears', 'show', 'emotion', 'screams', 'show', 'fear', 'shouting', 'show', 'dispute', 'violence', 'show', 'anger', 'naturally', 'joyce', 'short', 'story', 'lends', 'film', 'ready', 'made', 'structure', 'perfect', 'polished', 'diamond', 'small', 'changes', 'huston', 'makes', 'inclusion', 'poem', 'fit', 'neatly', 'truly', 'masterpiece', 'tact', 'subtlety', 'overwhelming', 'beauty']


In [10]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text()
    #
    # 2. Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))
    #
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    #
    # 6. Join the words back into one string separated by space,
    # and return the result.
    return( " ".join( meaningful_words ))

In [ ]:
clean_review = review_to_words( train["review"][0] )
print(clean_review)

naturally film main themes mortality nostalgia loss innocence perhaps surprising rated highly older viewers younger ones however craftsmanship completeness film anyone enjoy pace steady constant characters full engaging relationships interactions natural showing need floods tears show emotion screams show fear shouting show dispute violence show anger naturally joyce short story lends film ready made structure perfect polished diamond small changes huston makes inclusion poem fit neatly truly masterpiece tact subtlety overwhelming beauty


# Loop through and clean all of the training set at once

Sometimes it can be annoying to wait for a lengthy piece of code to run. It can be helpful to write code so that it gives status updates. To have Python print a status update after every 1000 reviews that it processes, try adding a line or two to the code above:

In [11]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size
print("Cleaning and parsing the training set movie reviews...\n")
# Initialize an empty list to hold the clean reviews
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



#Random Forest

In [ ]:
# Separate features and target
X_train = train.drop(columns=['id', 'sentiment', 'review'])
y_train = train['sentiment']
X_test = test.drop(columns=['id', 'review'])
y_test = test_y['sentiment']

In [ ]:
print("Training the random forest...")
# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100)

# Fit the forest to the training set, using the embeddings as
# features and the sentiment labels as the response variable
forest = forest.fit(X_train, y_train )

Training the random forest...


In [12]:
# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = []

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )



Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [ ]:
# Prediction
# Use the random forest to make sentiment label predictions
result = forest.predict(X_test)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "prob":result} )

# Use pandas to write the comma-separated output file
output.to_csv(f"{DATA_DIR}/split_1/mysubmission.csv", index=False, quoting=3 )

#Logistic Regression


In [14]:
def preprocess(data):
  # Get the number of reviews based on the dataframe column size
  num_reviews = data["review"].size
  print("Cleaning and parsing the data set movie reviews...\n")
  # Initialize an empty list to hold the clean reviews
  clean_data_reviews = []
  for i in range( 0, num_reviews ):
      # If the index is evenly divisible by 1000, print a message
      if( (i+1)%1000 == 0 ):
          print("Review %d of %d\n" % ( i+1, num_reviews ))
      clean_data_reviews.append( review_to_words( data["review"][i] ))
  return data

In [56]:
num_splits=5
for i in range(num_splits):
  train = pd.read_csv(f"{DATA_DIR}/split_{i+1}/train.csv")
  test = pd.read_csv(f"{DATA_DIR}/split_{i+1}/test.csv")

  train=preprocess(train)
  test=preprocess(test)

  # Separate features and target
  X_train = train.drop(columns=['id', 'sentiment', 'review'])
  y_train = train['sentiment']
  X_test = test.drop(columns=['id', 'review'])
  y_test = test_y['sentiment']



  # Logistic regression without penalty as baseline
  log_reg = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
  log_reg.fit(X_train, y_train)
  y_pred_proba = log_reg.predict_proba(X_test)[:, 1]
  auc_baseline = roc_auc_score(y_test, y_pred_proba)
  print(f"Baseline Logistic Regression AUC in split {i+1}: {auc_baseline:.3f}")


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [65]:
!pip install glmnet_python

ERROR: Could not find a version that satisfies the requirement glmnet_python (from versions: none)
ERROR: No matching distribution found for glmnet_python


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
# Initialize LogisticRegression with saga solver for elastic net


train = pd.read_csv(f"{DATA_DIR}/split_3/train.csv")

train=preprocess(train)

# Separate features and target
X_train = train.drop(columns=['id', 'sentiment', 'review'])
y_train = train['sentiment']


logreg = LogisticRegression(
    penalty='elasticnet',
    solver='saga',  # 'saga' is the only solver supporting elastic net in sklearn
    max_iter=1000  # Increase if convergence issues occur
)

# Set up parameter grid for cross-validation
param_grid = {
    'l1_ratio': [0.1],  # Similar to glmnet's alpha values
    'C': [10]  # Inverse of regularization strength, lower is more regularization
}

# Set up cross-validation with StratifiedKFold and AUC scoring
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=logreg,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=cv,
    verbose=1,
    n_jobs=-1
)

# Perform grid search
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best AUC Score:", grid_search.best_score_)


Cleaning and parsing the data set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [20]:
# Train the final model with the best parameters
final_model = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    l1_ratio=grid_search.best_params_['l1_ratio'],
    C=grid_search.best_params_['C'],
    max_iter=1000
)

final_model.fit(X_train, y_train)

LogisticRegression(C=10, l1_ratio=0.1, max_iter=1000, penalty='elasticnet',
                   solver='saga')

In [28]:
# Making Prediction
# Load test data
test = pd.read_csv(f"{DATA_DIR}/split_3/test.csv")
display(test)
test=preprocess(test)
X_test = test.drop(columns=['id', 'review'])

# Predict probabilities for the test data
predictions = final_model.predict_proba(X_test)[:, 1]  # Probabilities for positive sentiment


,id,review,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_1527,embedding_1528,embedding_1529,embedding_1530,embedding_1531,embedding_1532,embedding_1533,embedding_1534,embedding_1535,embedding_1536
0,29509,Basically the exact same movie as \House of Wa...,-0.021662,0.030561,-0.068984,0.004591,-0.022093,-0.017758,0.044535,0.001618,...,0.020464,-0.021783,-0.014876,-0.004850,-0.003669,0.012386,-0.010730,-0.017784,-0.000162,-0.021271
1,25369,"At the name of Pinter, every knee shall bow - ...",-0.024906,0.035320,-0.023498,0.011947,0.007341,-0.019141,0.038077,0.053156,...,0.032826,-0.003242,0.015298,-0.004045,-0.008185,0.041539,-0.004947,-0.043299,0.005533,-0.036875
2,17988,There is a point in the film where the female ...,0.012776,0.121584,-0.034502,-0.022849,-0.043452,0.018920,0.009379,0.038756,...,0.001451,0.005646,-0.026009,0.028284,-0.014947,0.031991,0.014866,0.007968,-0.026157,0.005738
3,34492,I was one of many that expected to see a glori...,-0.034762,0.036142,-0.060874,0.032454,0.024188,-0.047287,-0.006651,0.049782,...,-0.012883,-0.029428,-0.016890,-0.016001,-0.014688,0.045430,0.002058,0.003307,0.011828,-0.022834
4,35433,"Being a slasher film aficionado, I typically w...",0.012299,0.091021,-0.057493,-0.020125,-0.001237,0.011167,0.042894,0.007278,...,-0.019436,-0.028521,-0.041263,-0.008347,0.001180,-0.007334,-0.003931,0.026609,-0.007236,-0.006343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,23538,I must not have seen the same movie as the one...,-0.023828,0.063136,-0.031670,-0.024450,-0.035324,0.002988,0.008260,0.007442,...,-0.005161,0.009503,0.001033,-0.012054,-0.003940,0.026645,0.010988,-0.016964,0.023930,-0.040501
24996,37764,This movie serves as a timely warning to anyon...,-0.033881,0.066702,-0.074066,-0.010126,-0.013265,-0.036715,0.029934,0.015391,...,-0.030940,-0.013073,0.017099,-0.007841,0.003689,0.029298,0.031470,-0.006533,0.027152,-0.026914
24997,44137,Kusturika made it again. Another masterpiece. ...,0.012671,0.066837,-0.067419,-0.022566,-0.004132,0.000491,0.016443,0.056070,...,0.038778,0.039190,-0.014539,-0.027914,-0.010652,0.033710,0.014527,0.006133,-0.031018,-0.045229
24998,24602,This movie was fabulous. It is definitely a to...,-0.041308,0.070298,-0.081323,-0.014908,-0.000524,-0.027338,0.017496,0.041956,...,0.031690,-0.037559,-0.028722,-0.017050,0.015856,-0.007080,-0.011806,-0.000072,-0.008425,-0.034613


Cleaning and parsing the data set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [30]:
# Saving Predictions

y_test = pd.read_csv(f'{DATA_DIR}/split_3/test_y.csv')['sentiment']
display(y_test)

    # Step 2: Save predictions to mysubmission1.txt
submission = pd.DataFrame({
    'id': test['id'],
    'prob': predictions
})

submission.to_csv(f'{DATA_DIR}/split_3/mysubmission2.txt', index=False)




            # Evaluation. #### To be removed before submission####

,sentiment
0,1
1,0
2,0
3,1
4,0
...,...
24995,0
24996,0
24997,1
24998,1


In [31]:
# Calculate AUC on the test data
test_auc_score = roc_auc_score(y_test, predictions)
print("Best AUC Score on Test Data:", test_auc_score)

Best AUC Score on Test Data: 0.9867907678840266
